In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.integrate import odeint
from numpy import linalg as LA
from numpy.linalg import inv
from math import exp

In [2]:
def get_yeastract_data(yeastract_csv_filename, gene_to_orf_filename="tftoorf.csv", as_orf=False):
    '''
    Returns Yeastract network data as a pandas DataFrame.
    '''
    def tf_to_gene(tf_name):
        '''
        Removes trailing "p" from transcription factor name and turn into uppercase.
        '''
        if tf_name[-1] == 'p':
            return tf_name.upper()[:-1]
        else:
            return tf_name
    
    yeastract_data = pd.read_csv(yeastract_csv_filename, sep= ";")
    gene_to_orf_df = pd.read_csv(gene_to_orf_filename)
    
    tf_list = list(yeastract_data['6855'])
    gene_list = list(yeastract_data.columns.values)[1:]
    
    # Fix dataframe so it has right things as rownames.
    yeastract_data = yeastract_data.rename(index=yeastract_data['6855'])
    yeastract_data = yeastract_data.iloc[:, 1:]
    
    tf_list = [tf_to_gene(tf) for tf in tf_list]
    if as_orf:
        gene_to_orf = dict(zip(list(gene_to_orf_df.iloc[:,0]), list(gene_to_orf_df.iloc[:,2])))
        
        rownames = [gene_to_orf[tf] for tf in tf_list]
        # Change to TF if possible, otherwise keep same name.
        colnames = [gene_to_orf.get(gene, gene) for gene in gene_list]
    else:
        rownames = tf_list
        colnames = gene_list
    
    final_data = pd.DataFrame(np.array(yeastract_data), index=rownames, columns=colnames)
    
    return final_data

def get_gasch_data(gasch_data_filename="complete_dataset_gasch.txt", 
                   orf_to_gene_filename="orfname_time_course_fixed.csv",
                   supset=None ,
                   as_orf=False):
    '''
    Returns heat shock time-course data from Gasch as DataFrame.
    '''
    gasch_data = pd.read_csv(gasch_data_filename, sep="\t")
#    gasch_data = gasch_data.iloc[:,:11] # Filter only heat-shock data from first experiment (hs-1).
    
    if not as_orf:
        # Rename as genes.
        orf_to_gene_df = pd.read_csv(orf_to_gene_filename)
        orfs = list(orf_to_gene_df.iloc[:,0])
        genes = list(orf_to_gene_df.iloc[:,3])
        
        # Remove unknowns, keep original name.
        genes = [gene if gene != 'Unknown' else orfs[i] for i, gene in enumerate(genes)]
        orf_to_gene = dict(zip(orfs, genes))
        
        orf_list = list(gasch_data["UID"])
        gene_names = [orf_to_gene.get(orf, orf) for orf in orf_list]
        #print(gene_names)
        gasch_data["UID"] = gene_names
        gasch_data = gasch_data.rename(index=gasch_data["UID"])
        gasch_data = gasch_data.iloc[:, 1:]
        if supset != None:
            to_delete = set(gene_names)-set(supset)
            to_delete_list = list(to_delete)
            gasch_data = gasch_data.drop(to_delete_list)
    return gasch_data
    

def squarify(df, sort=False):
    '''
    Turns Yeastract DataFrame into a square dataframe (i.e. adjacency matrix).
    '''
    cols=list(df.columns)
    rows=list(df.index)
    d = []
    for x in cols:
        if x in rows:
            d.append(list(df.loc[x]))
        else:
            d.append([0]*len(cols))
    A = np.array(d)
    return pd.DataFrame(A, index=cols, columns=cols)

def reorder(target_orf, adj_matrix):
    A = target_orf #input the list of target orf name
    B = adj_matrix #input the adj matrix

    #First we want to extract the header from the adj matrix and compare them with the list

    B_header = list(B)

    AB_diff = list(set(B_header) - set(A))
    #print(set(A)<set(B_header))
    A_total = A + AB_diff
    B = B[A_total]
    B = B.reindex(A_total)
    return B

In [3]:
yeastract_data_activation = get_yeastract_data("matrix_activator.csv","tftoorf.csv")
yeastract_data_inhibition = get_yeastract_data("matrix_inhibitor.csv","tftoorf.csv")
adj_mat_activation = squarify(yeastract_data_activation)
adj_mat_inhibition = squarify(yeastract_data_inhibition)
adj_matrix=adj_mat_activation-adj_mat_inhibition
cols = list(adj_mat_activation.columns)
gasch_data = get_gasch_data("complete_dataset_gasch.txt","orfname_time_course_fixed.csv",cols)
adj_matrix = reorder(list(gasch_data.index),adj_matrix)
gasch_esr = get_gasch_data("figure3_gasch_paper.cdt","orfname_time_course_fixed.csv",cols)

In [37]:
esr_genes=list(gasch_esr.index)

In [38]:
gasch_data_cols=gasch_data.columns
gasch_data_cols[0:17]

Index(['NAME', 'GWEIGHT', 'Heat Shock 05 minutes hs-1',
       'Heat Shock 10 minutes hs-1', 'Heat Shock 15 minutes hs-1',
       'Heat Shock 20 minutes hs-1', 'Heat Shock 30 minutes hs-1',
       'Heat Shock 40 minutes hs-1', 'Heat Shock 60 minutes hs-1',
       'Heat Shock 80 minutes hs-1', 'Heat Shock 000 minutes hs-2',
       'Heat Shock 000 minutes  hs-2', 'Heat Shock 000 minutes  hs-2.1',
       'Heat Shock 005 minutes  hs-2', 'Heat Shock 015 minutes  hs-2',
       'Heat Shock 030inutes  hs-2', 'Heat Shock 060 minutes  hs-2'],
      dtype='object')

In [187]:
#These transcription factors are the ones we expect to turn on the heat shock response.
heat_shock_starters=["HSF1","MSN2","MSN4","RLM1","SWI4"]

In [4]:
def num_to_rgb(num,maxi):
    if num > 0:
        return [int(255*num/maxi), 0, 0]
    elif num < 0:
        return [0,int(-255*num/maxi),0]
    elif num ==0:
        return [0,0,0]
    else:
        return [0,0,255]
def make_plot(rawdata,filename,maxi=8,pixperpoint=10):
    numrows=len(rawdata)
    numcols=len(rawdata[0])
    raw_pixel_data = np.zeros([numrows,pixperpoint*numcols,3],dtype=np.uint8)
    for i in range(numrows):
        for j in range(numcols):
            for k in range(pixperpoint):
                raw_pixel_data[i,pixperpoint*j+k] = num_to_rgb(rawdata[i,j], maxi)
    img = Image.fromarray(raw_pixel_data, 'RGB')
    img.save(filename)

In [39]:
esr_raw_data=gasch_data.loc[esr_genes]
esr_raw_data_heat_shock=esr_raw_data[gasch_data_cols[2:10]]
esr_raw_data_heat_shock_array=np.array(esr_raw_data_heat_shock)
make_plot(esr_raw_data_heat_shock_array,'esr_heat_shock.png',8,10)

In [189]:
for h in heat_shock_starters:
    print(h,cols.index(h))

HSF1 1807
MSN2 4038
MSN4 3154
RLM1 5525
SWI4 1405


In [7]:
A=np.transpose(np.array(adj_matrix))
u=np.zeros(len(cols))
u[1807] = 1
u[4038] = 1
u[3154] = 1
u[5525] = 1
u[1405] = 1
y0=np.zeros(len(cols))
kf=0.5

In [13]:
eigvals,eigvects = LA.eig(A-kf*np.identity(len(cols)))
V = np.transpose(eigvects)
Vinv = inv(V)
Vhat= Vinv@u
md=[]
mdtemp=[]
for t in [0,5,10,15,20,30,40,60,80]:
    for lamda in eigvals:
# We're going to have a huge problem here where zero eigenvalues are not recognized as
# properly being zero because of roundoff error. This is going to add humongous entries
# to the diagonal matrix, which is bad
        if lamda == 0:
            mdtemp.append(t)
        else:
            mdtemp.append((exp(lamda*t))/lamda)
    md.append(mdtemp)
    mdtemp=[]
time_course=[]
for tstep in md:
    time_course.append(V @ np.diag(tstep) @ Vhat)
time_course=np.transpose(time_course)
time_course=time_course.real

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':
